In [0]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import pickle

from keras.preprocessing import image
from HOG_Processor import HOGExtractor
from Sliding_Window import CarFinder
from SVM_Classifier import SVM
from Util import process_dataset, draw_box, draw_label
from CNN_Classifier import CNN

In [0]:
def main(image_file, vehicle_path, non_vehicle_path, CNN_Train_path, CNN_Test_Path):
    
    # Process the dataset images, extract their hog features
    process_dataset(1, vehicle_path)
    process_dataset(0, non_vehicle_path)

    # Train SVM
    positive_datapaths = ['HogData/positive.pickle']
    negative_datapaths = ['HogData/negative.pickle']

    svm = SVM(positive_datapaths, negative_datapaths)
    svm.train_svc()
    svm.train_linearSvc()

    # Store the trained SVM
    with open("TrainedSvm/svm" + '.pickle', 'wb') as handle:
        pickle.dump(svm, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Find vehicle fragmentation by the trained SVM
    with open("TrainedSvm/svm" + '.pickle', 'rb') as handle:
        svm = pickle.load(handle)

    extractor = HOGExtractor(64, 12, 8, 2, True)
    carFinder = CarFinder(extractor, svm, 125, 10)
    carFinder.find_car(image_file, 'svc')
    carFinder.find_car(image_file, 'linearSvc')

    # Combine the fragmentation into a whole vehicle image
    svc_clusters, svc_cars = carFinder.find_group(image_file, 'HogData/validWindows_svc.pickle')
    linearSvc_clusters, linearSvc_cars = carFinder.find_group(image_file, 'HogData/validWindows_linearSvc.pickle')

    with open("HogData/validWindows_svc.pickle", 'rb') as handle:
        valid_windows_svc = pickle.load(handle)
    with open("HogData/validWindows_linearSvc.pickle", 'rb') as handle:
        valid_windows_linearSvc = pickle.load(handle)

    # Draw all the boxex detected as vehicle
    valid_windows_svc = draw_box(image_file, valid_windows_svc)
    valid_windows_linearSvc = draw_box(image_file, valid_windows_linearSvc)
    cv.imwrite('results/valid_windows_svc.jpg', valid_windows_svc)
    cv.imwrite('results/valid_windows_linearSvc.jpg', valid_windows_linearSvc)

    # Draw the grouped boxes
    detected_svc = draw_box(image_file, svc_clusters)
    detected_linearSvc = draw_box(image_file, linearSvc_clusters)
    cv.imwrite('results/detected_svc.jpg', detected_svc)
    cv.imwrite('results/detected_linearSvc.jpg', detected_linearSvc)

    #Start training CNN
    cnn = CNN(CNN_Train_path, CNN_Test_Path)
    cnn.train_cnn_with_vgg16()
    
    # Classify the vehicle detected with SVM (Linear Kernel)
    for i in range(len(linearSvc_cars)):
        
      # Preprocess Image
      test_image = image.img_to_array(cv.resize(linearSvc_cars[i],(64, 64)))
      test_image = np.expand_dims(test_image, axis = 0)
    
      result = cnn.vgg.predict(test_image)
      tag = cnn.classes[np.argmax(result)]
      
      # Draw label on the detected box
      detected_linearSvc = draw_label(detected_linearSvc, tag, linearSvc_clusters[i])
      
    # Classify the vehicle detected with SVM (Radial Kernel)
    for j in range(len(svc_cars)):
        
      # Preprocess Image  
      test_image = image.img_to_array(cv.resize(svc_cars[j],(64, 64)))
      test_image = np.expand_dims(test_image, axis = 0)
    
      result = cnn.vgg.predict(test_image)
      tag = cnn.classes[np.argmax(result)]

      # Draw label on the detected box
      detected_svc = draw_label(detected_svc, tag, svc_clusters[j])
    
    cv.imwrite("resultB.jpg", detected_linearSvc)
    cv.imwrite("resultA.jpg", detected_svc)

In [ ]:
main('random.JPG', "vehicle", "non-vehicle", "TRAIN", "TEST")